In [1]:
import pandas as pd
import numpy as np
import spacy
import utils as ut # esta librería tiene funciones para poder obtener un procesamiento del <T,H>
import time
from scipy.stats import wasserstein_distance
import mutual_info as mi
import sys

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
dirdata = "../OPENAI/"

In [3]:
import conceptnet_lite
conceptnet_lite.connect(dirdata+"data/conceptnet.db")
from conceptnet_lite import Label, edges_for, edges_between

In [4]:
## esta función revisa hiperonimia, sinonimia entre otras.
def encontrar_relaciones(relaciones):
    borrar=set()
    borrar_i=set()
    for r in relaciones:
        index=r[0]
        c=r[1]
        wt=str(index).split("{")[1].split(",")[0]
        wh=str(c).split("{")[1].split(",")[0]    
        try:
            bandera=0
            concepts_wt = Label.get(text=wt, language='en').concepts
            concepts_wh = Label.get(text=wh, language='en').concepts
            for e in edges_between(concepts_wt, concepts_wh, two_way=False,relation="is_a"):
                print(wt," is_a ",wh)
                borrar.add(c)
                borrar_i.add(index)
                bandera=1
                break
            if bandera==0:
                for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="synonym"):
                    print(wh," synonym ",wt)
                    borrar.add(c)
                    borrar_i.add(index)
                    bandera=1
                    break
            if bandera==0:
                for e in edges_between(concepts_wh,concepts_wt, two_way=False,relation="has_a"):
                    print(wh," has_a ",wt)
                    borrar.add(c)
                    borrar_i.add(index)
                    bandera=1
                    break
            if bandera==0:
                for e in edges_between(concepts_wt,concepts_wh, two_way=False,relation="derived_from"):
                    print(wt," derivado_from ",wh)
                    borrar.add(c)
                    borrar_i.add(index)
                    bandera=1
                    break
            if bandera==0:
                for e in edges_between(concepts_wt,concepts_wh, two_way=False,relation="used_for"):
                    print(wt," used_for ",wh)
                    borrar.add(c)
                    bandera=1
                    borrar_i.add(index)
                    break
            if bandera==0:
                for e in edges_between(concepts_wt,concepts_wh, two_way=False,relation="HasProperty"):
                    bandera=1
                    print(wt," HasProperty ",wh)
                    borrar.add(c)
                    borrar_i.add(index)
                    break
        except:
            a=0
    return list(borrar),list(borrar_i)

In [5]:
def encontrar_relaciones_contrarias(relaciones):
    antonyms=[]
    borrar=[]
    borrar_i=[]
    for r in relaciones:
        index=r[0]
        c=r[1]
        wt=str(index).split("{")[1].split(",")[0]
        wh=str(c).split("{")[1].split(",")[0]
        try:
            bandera=0
            concepts_wt = Label.get(text=wt, language='en').concepts
            concepts_wh = Label.get(text=wh, language='en').concepts
            for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="antonym"):
                print(wt," antonym ",wh)
                # borrar.append(c)
                # borrar_i.append(index)
                antonyms.append(wt)
                bandera=1
                break
            if bandera==0:
                for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="distinct_from"):
                    print(wh," distinct from ",wt)
                    # borrar.append(c)
                    bandera=1
                    # borrar_i.append(index)
                    antonyms.append(wt)
                    break
            if bandera==0:
                for e in edges_between(concepts_wh, concepts_wt, two_way=False,relation="is_a"):
                    print(wt," is_a (ant) ",wh)
                    bandera=1
                    # borrar.add(c)
                    # borrar_i.add(index)
                    break
            if bandera==0:
                for e in edges_between(concepts_wt, concepts_wh, two_way=False,relation="has_a"):
                    print(wh," has_a (ant)",wt)
                    bandera=1
                    #borrar.add(c)
                    #borrar_i.add(index)
                    break
            if bandera==0:
                for e in edges_between( concepts_wh, concepts_wt, two_way=False,relation="derived_from"):
                    bandera=1
                    print(wh," derivado_from (ant)",wt)
                    #borrar.add(c)
                    #borrar_i.add(index)
                    break
            if bandera==0:
                for e in edges_between( concepts_wh, concepts_wt,two_way=False,relation="used_for"):
                    bandera=1
                    print(wh," used_for (ant)",wt)
                    #borrar.add(c)
                    #borrar_i.add(index)
                    break
            if bandera==0:
                for e in edges_between(concepts_wh,concepts_wt, two_way=False,relation="HasProperty"):
                    bandera=1
                    print(wh," HasProperty ",wt)
                    borrar.add(c)
                    borrar_i.add(index)
                    break
        except:
            a=0
    return borrar,borrar_i,antonyms

In [6]:
def encontrar_relaciones_cercanas(indexes,columnas):
    borrar=[]
    borrar_i=[]
    related=[]
    for index in indexes:
        for c in columnas:
            wt=str(index).split("{")[1].split(",")[0]
            wh=str(c).split("{")[1].split(",")[0]
            try:
                for e in edges_for(Label.get(text=wh, language='en').concepts, same_language=True):
                    if (wt == e.start.text or wt == e.end.text) and (e.relation.name in ["related_to","similar_to"]):
                        if "related_to" ==e.relation.name:
                            if wt== e.start.text:
                                if e.end.text==wh:
                                    print(wt," related_to ",wh)
                                    borrar.append(c)
                                    borrar_i.append(index)
                                    related.append(wt)
                            else:
                                if wt== e.end.text:
                                    print(wh," related_to ",wt)
                                    borrar.append(c)
                                    borrar_i.append(index)
                                    related.append(wt)
                        elif "similar_to" ==e.relation.name:
                            if wh== e.start.text:
                                if e.end.text==wt:
                                    print(wh," similar_to ",wt)
                                    borrar.append(c)
                                    borrar_i.append(index)
                                    related.append(wt)
                            else:
                                if e.start.text==wt:
                                    print(wt," similar_to ",wh)
                                    borrar.append(c)
                                    borrar_i.append(index)
                                    related.append(wt)
            except:
                a=0
    return borrar,borrar_i,related

In [7]:
def obtener_distancia(texto_v,hipotesis_v,texto_t,texto_h,b_col,b_index):
    lista_l=[]
    for i in range(len(texto_t)):
        lista=[]
        for j in range(len(texto_h)):
            lista.append(np.linalg.norm(texto_v[i] - hipotesis_v[j]))#*wasserstein_distance(texto_2[i],hipotesis_2[j]))
        lista_l.append(lista)
    df_distEuc=pd.DataFrame(lista_l,index=texto_t,columns=texto_h)
    df_distEuc=df_distEuc.drop(b_col[1:],axis=1)
    df_distEuc=df_distEuc.drop(b_index[1:],axis=0)
    return df_distEuc

In [8]:
def wasserstein_mutual_inf(texto_v,hipotesis_v,texto_t,texto_h):  
    lista_l=[]
    lista_muinfor=[]   
    for i in range(len(texto_t)):
        lista=[]
        lista_mu=[]
        for j in range(len(texto_h)):
            lista.append(wasserstein_distance(texto_v[i],hipotesis_v[j]))
            lista_mu.append(mi.mutual_information_2d(np.array(texto_v[i]),np.array(hipotesis_v[j])))
        lista_l.append(lista)
        lista_muinfor.append(lista_mu)
    DFmearth=pd.DataFrame(lista_l,index=texto_t,columns=texto_h)
    DFmutual_inf=pd.DataFrame(lista_muinfor,index=texto_t,columns=texto_h)
    return DFmearth,DFmutual_inf

In [9]:
def entropia(X):
    """Devuelve el valor de entropia de una muestra de datos""" 
    probs = [np.mean(X == valor) for valor in set(X)]
    return round(sum(-p * np.log2(p) for p in probs), 3)

In [10]:
nlp = spacy.load("en_core_web_md") # modelo de nlp

In [11]:
ut.load_vectors_in_lang(nlp,"./data/numberbatch-en-17.04b.txt") # carga de vectores en nlp.wv


In [12]:
archivo="DEV_1.csv"

In [13]:
prueba=pd.read_csv(dirdata+"data/RTE3/"+archivo)
#prueba=pd.read_csv(dirdata+"data/"+sys.argv[1])

textos = prueba["sentence1"].to_list()       # almacenamiento en listas
hipotesis = prueba["sentence2"].to_list()

In [14]:
sumas=[]
distancias=[]
entropias=[]
etiquetas=[]
mearts=[]
mutinf=[]
max_info=[]
list_antonimos=[]
listas_malign=[]
lista_anto=[]
lista_related=[]
lista_relatedT=[]
diferencias=[]
similitud_faltantes=[]

## Producto punto de palabras

In [15]:
word1 = "claude chabrol (born june 24, 1930) is a french movie director and has become well-known in the 40 years since his first film, le beau serge , for his chilling tales of murder, including le boucher"
word2 = "Female mosquitoes spread malaria"
t_vectors_n=ut.get_matrix_rep(word1,nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
h_vectors_n=ut.get_matrix_rep(word2,nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
t_clean=ut.get_words_rep(word1,nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
normalt_clean=ut.get_words(word1,nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
h_clean=ut.get_words_rep(word2,nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
# Obtención de matriz de alineamiento
ma=np.dot(t_vectors_n,h_vectors_n.T)
ma=pd.DataFrame(ma,index=t_clean,columns=h_clean)

ma

,"female{female,ADJ}","mosquitoes{mosquito,NOUN}","spread{spread,VERB}","malaria{malaria,NOUN}"
"claude{claude,PROPN}",0.089979,-0.053149,-0.205454,-0.051530
"chabrol{chabrol,PROPN}",0.000000,0.000000,0.000000,0.000000
"born{bear,VERB}",0.086176,-0.106916,-0.067275,-0.057872
"june{june,PROPN}",-0.015725,-0.106503,-0.003842,0.023486
"24{24,NUM}",0.015911,-0.131447,-0.010650,0.021100
"1930{1930,NUM}",0.144675,-0.097227,-0.038802,0.048092
"is{be,AUX}",0.128430,0.186277,0.126122,0.127613
"a{a,DET}",0.259288,0.201673,0.119810,0.110989
"french{french,ADJ}",0.115437,0.093472,0.108986,-0.058617
"movie{movie,NOUN}",0.140620,0.084111,0.007349,0.002181


In [16]:
normalt_clean

['claude',
 'chabrol',
 'born',
 'june',
 '24',
 '1930',
 'is',
 'a',
 'french',
 'movie',
 'director',
 'and',
 'has',
 'become',
 'well',
 'known',
 'in',
 'the',
 '40',
 'years',
 'since',
 'his',
 'first',
 'film',
 'le',
 'beau',
 'serge',
 'for',
 'his',
 'chilling',
 'tales',
 'of',
 'murder',
 'including',
 'le',
 'boucher']

In [67]:
def reform_sentence(frase):
    frase_m=[]
    b=1
    for i in range(len(frase)-1):
        word_h=frase[i]+"_"+frase[i+1]
        try:
            if word_h in nlp.wv:
                frase_m.append(word_h)
                b=0
            elif b==0:
                b=1
            else:
                frase_m.append(frase[i])
        except:
            pass
    return " ".join(frase_m) + " " + frase[i+1]

In [68]:
#nlp.wv["well_known"]

In [69]:
rec=reform_sentence(normalt_clean)
rec

'claude chabrol born june 24 1930 is a french movie director and has become well_known in the 40 years since his first film le beau serge for his chilling tales of murder including le boucher'

In [51]:
t_clean

['claude{claude,PROPN}',
 'chabrol{chabrol,PROPN}',
 'born{bear,VERB}',
 'june{june,PROPN}',
 '24{24,NUM}',
 '1930{1930,NUM}',
 'is{be,AUX}',
 'a{a,DET}',
 'french{french,ADJ}',
 'movie{movie,NOUN}',
 'director{director,NOUN}',
 'and{and,CCONJ}',
 'has{have,AUX}',
 'become{become,VERB}',
 'well{well,ADV}',
 'known{know,VERB}',
 'in{in,ADP}',
 'the{the,DET}',
 '40{40,NUM}',
 'years{year,NOUN}',
 'since{since,SCONJ}',
 'his{his,PRON}',
 'first{first,ADJ}',
 'film{film,NOUN}',
 'le{le,PROPN}',
 'beau{beau,PROPN}',
 'serge{serge,PROPN}',
 'for{for,ADP}',
 'his{his,PRON}',
 'chilling{chilling,ADJ}',
 'tales{tale,NOUN}',
 'of{of,ADP}',
 'murder{murder,NOUN}',
 'including{include,VERB}',
 'le{le,X}',
 'boucher{boucher,NOUN}']

In [7]:
textos=[]
hipotesis=[]

In [8]:
textos.append(word1)
hipotesis.append(word1)

In [17]:
def bi_grams(texto1,texto2):
    gramsT=set()
    gramsH=set()
    palabras = texto1.split()
    for i in range(len(palabras)-1):
        gramsT.add((palabras[i],palabras[i+1]))
    palabras = texto2.split()
    for i in range(len(palabras)-1):
        gramsH.add((palabras[i],palabras[i+1]))
    return len(gramsT.intersection(gramsH))/len(gramsT)

In [18]:
bi_grams("Hugh was also very supportive of Mustang's plan to become Fuhrer, doing whatever he can to, as he put it, to help him from the bottom to the top.","Hugh assisted Mustang.")

0.0

In [43]:
#import textacy

In [45]:
from textacy import extract, text_stats
#textacy.set_doc_extensions("extract")

In [49]:
#from __future__ import unicode_literals
#import spacy,en_core_web_md
#import textacy
#nlp = en_core_web_md.load()
#sentence = 'The author is writing a new book.'
#pattern = r'The'
#doc = textacy.make_spacy_doc(sentence, lang='en_core_web_md')
#print(doc)
#lists = textacy.extract.regex_matches(doc,pattern)
#for list in lists:
#    print(list)

The author is writing a new book.
The


In [50]:
from textacy import extract
list(extract.entities(doc, include_types={"PERSON", "LOCATION"}))

TypeError: 'spacy.tokens.span.Span' object is not callable

In [22]:
import textacy
from textacy import extract
text = "Female mosquitoes become infected with the malaria parasite when they draw blood from humans with malaria. The insects can then pass this on to other humans they bite, but do not get sick themselves."
doc = textacy.make_spacy_doc(text, lang="en_core_web_md")
print(doc._.preview)

Doc(37 tokens: "Female mosquitoes become infected with the mala...")


In [23]:
doc

Female mosquitoes become infected with the malaria parasite when they draw blood from humans with malaria. The insects can then pass this on to other humans they bite, but do not get sick themselves.

In [24]:
list(extract.entities(doc, include_types={"PERSON", "ORG", "GPE", "LOC"}))

[]

In [26]:
assle=list(extract.subject_verb_object_triples(doc))

In [35]:
for a in assle:
    #sujeto, verbo y objeto
    #print(a[0],a[1],a[2])
    for e in a[1]:
        print(a[0][0],e,a[2][0])

they draw blood
insects can this
insects pass this
insects do themselves
insects not themselves
insects get themselves


In [58]:
text = "Female mosquitoes become infected with the malaria parasite when they draw blood from humans with malaria. The insects can then pass this on to other humans they bite, but do not get sick themselves."
doc = textacy.make_spacy_doc(text, lang="en_core_web_md")

In [59]:
for t in doc:
    print(t,t.pos_)

Female ADJ
mosquitoes NOUN
become VERB
infected VERB
with ADP
the DET
malaria NOUN
parasite NOUN
when SCONJ
they PRON
draw VERB
blood NOUN
from ADP
humans NOUN
with ADP
malaria NOUN
. PUNCT
The DET
insects NOUN
can AUX
then ADV
pass VERB
this PRON
on ADP
to ADP
other ADJ
humans NOUN
they PRON
bite VERB
, PUNCT
but CCONJ
do AUX
not PART
get VERB
sick ADJ
themselves PRON
. PUNCT


In [16]:
doc

The Kinston Indians are a minor league baseball team in Kinston, North Carolina. The team, a Class A affiliate of the Cleveland Indians, plays in the Carolina League.

In [13]:
from functools import partial
terms = list(extract.terms(doc,
    ngs=partial(extract.ngrams, n=2, include_pos={"NOUN", "ADJ", "VERB","ADV","PART"}),
    ents=partial(extract.entities, include_types={"PERSON", "ORG", "GPE", "LOC"}),
    dedupe=False))
terms

[minor league,
 league baseball,
 baseball team,
 Kinston,
 North Carolina,
 Cleveland Indians,
 Carolina League]

In [37]:
a=list(extract.subject_verb_object_triples(doc))

In [54]:
for e in a:
    trip = ""
    for e1 in e[0]:
        trip += str(e1) + " "
    for e1 in e[1]:
        trip += str(e1) + " "
    for e1 in e[2]:
        trip += str(e1) + " "
    print(trip)

they draw blood 
insects can pass this 
insects do not get themselves 


In [ ]:

def get_grams_entities(texto):
    doc = textacy.make_spacy_doc(texto, lang="en_core_web_md")
    terms = list(extract.terms(doc,
    ngs=partial(extract.ngrams, n=2, include_pos={"NOUN", "ADJ", "VERB","ADV","PART"}),
    ents=partial(extract.entities, include_types={"PERSON", "ORG", "GPE", "LOC"}),
    dedupe=False))
    a = list(extract.subject_verb_object_triples(doc))
    for e in a:
        trip = ""
        for e1 in e[0]:
            trip += str(e1) + " "
        for e1 in e[1]:
            trip += str(e1) + " "
        for e1 in e[2]:
            trip += str(e1) + " "
        terms.append(trip)
    return terms

In [60]:
print(str(terms1[0]))

minor league baseball


In [14]:
terms1 = list(extract.terms(doc,
    ngs=partial(extract.ngrams, n=3, include_pos={"NOUN", "ADJ", "VERB","ADV","PART"}),
    dedupe=True))
terms1

[minor league baseball, league baseball team]

In [15]:
terms2 = list(extract.terms(doc,
    ngs=partial(extract.ngrams, n=4, include_pos={"NOUN", "ADJ", "VERB","ADV","PART"}),
    dedupe=True))
terms2

[minor league baseball team]

In [18]:
terms.extend(terms1)

In [19]:
terms

[minor league,
 league baseball,
 baseball team,
 Kinston,
 North Carolina,
 Cleveland Indians,
 Carolina League,
 minor league baseball,
 league baseball team]

# Proceso de representación de relaciones de T -> H

In [42]:
inicio = time.time()
for i in range(len(textos)):
#for i in range(1):
    print(i)

    # Obtención de vectores densos de word embeddings
    t_vectors=ut.get_matrix_rep(textos[i],nlp,pos_to_remove=['PUNCT'], normed=False,lemmatize=False)
    h_vectors=ut.get_matrix_rep(hipotesis[i],nlp,pos_to_remove=['PUNCT'], normed=False,lemmatize=False)
    t_vectors_n=ut.get_matrix_rep(textos[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
    h_vectors_n=ut.get_matrix_rep(hipotesis[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
    t_clean=ut.get_words_rep(textos[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
    h_clean=ut.get_words_rep(hipotesis[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)


    # Obtención de matriz de alineamiento
    ma=np.dot(t_vectors_n,h_vectors_n.T)
    #print(len(t_vectors_n),len(h_vectors_n),len(t_clean),len(h_clean))
    # Obtención de matriz de move earth y mutual information
    m_earth,m_mi=wasserstein_mutual_inf(t_vectors_n,h_vectors_n,t_clean,h_clean)
    ma=pd.DataFrame(ma,index=t_clean,columns=h_clean)
    print(ma)
    print(m_earth)
    print(m_mi)
    ###### BORRADO DE COSAS QUE NO OCUPO, SOLO NOS QUEDAMOS CON INFORMACIÓN DE TIPOS DE PALABRA: NOUN, VERB, ADJ Y ADV
    # TAMBIÉN OMITIMOS EL VERBO BE DEBIDO A QUE POR LO REGULAR SE UTILIZA COMO AUXILIAR Y ES UN VERBO COPULATIVO
    # sirve para construir la llamada predicación nominal del sujeto de una oración: 
    # #el sujeto se une con este verbo a un complemento obligatorio llamado atributo que por lo general determina 
    # alguna propiedad, estado o equivalencia del mismo, por ejemplo: "Este plato es bueno". "Juan está casado".
     
    # Inicio de la lista de índices para el proceso de reducción de matriz
    b_col=[0]
    b_index=[0]
    
    # listado de indices de las palabras de T(filas) y de H (col)
    col=ma.columns
    indexes=ma.index
    ma
    print(ma)
    print(m_earth)
    print(m_mi)
    ### PROCESO DE REDUCCIÓN DE MATRIZ CON CONCEPNET
    # primer paso de eliminación de datos en la matriz de alineamiento
    borrar=[]
    for c in col:
        if "{null," in str(c) or "{be,VERB" in str(c) or (",NUM"  not in str(c) and "not," not in str(c) and "PRON" not in str(c) and "NOUN" not in str(c) and "VERB" not in str(c) and "ADJ" not in str(c) and "ADV" not in str(c)):
            borrar.append(c)        
        elif str(c) in indexes:
            borrar.append(c)        
    borrar_i=[]
    for index in indexes:
        if "{null," in str(index) or "{be,VERB" in str(index) or (",NUM"  not in str(index) and "not," not in str(index) and "PRON" not in str(index) and "NOUN" not in str(index) and "VERB" not in str(index) and "ADJ" not in str(index) and "ADV" not in str(index)):
            borrar_i.append(index) 
        elif str(index) in col:
            borrar_i.append(index) 
    ma=ma.drop(borrar,axis=1)
    ma=ma.drop(borrar_i,axis=0)
    m_earth=m_earth.drop(borrar,axis=1)
    m_earth=m_earth.drop(borrar_i,axis=0)
    m_mi=m_mi.drop(borrar,axis=1)
    m_mi=m_mi.drop(borrar_i,axis=0)

    b_col.extend(borrar)
    b_index.extend(borrar_i)
    
    # ELIMINAMOS INFORMACIÓN DONDE SE CORRESPONDAN EN LEMMAS, TENGA UN PRODUCTO IGUAL A 1 Y SEAN IGUALES LOS INDICES Y COLUMNAS
    # SI EL VALOR ES IGUAL A 1 SIGNIFICA QUE ES LA MISMA PALABRA, O SON SINONIMOS
    borrar=[]
    borrar_i=[]
    col=ma.columns
    for index,strings in ma.iterrows():
        lema_i=str(index).split("{")[1].split(",")[0]
        for c in col:
            if index==c:
                borrar_i.append(index)
                borrar.append(c)
            # if strings[c]>=1:
            #     borrar_i.append(index)
            #     borrar.append(c)
            lema_c=str(c).split("{")[1].split(",")[0]
            if lema_i == lema_c:
                borrar_i.append(index)
                borrar.append(c)
    ma=ma.drop(borrar,axis=1)
    ma=ma.drop(borrar_i,axis=0)
    m_earth=m_earth.drop(borrar,axis=1)
    m_earth=m_earth.drop(borrar_i,axis=0)
    m_mi=m_mi.drop(borrar,axis=1)
    m_mi=m_mi.drop(borrar_i,axis=0)
    
    b_col.extend(borrar)
    b_index.extend(borrar_i)



    #primera vuelta ---------------------------------------------------------------------------------
    # #PARA REVISAR SI EXISTEN RELACIONES DE SIMILITUD SEMÁNTICA A TRAVÉS DEL USO DE CONCEPNET
    d=[]
    d1=[]
    pasada=0
    #while len(ma.index)>0 and len(ma.columns)>0 and pasada<2:
    while len(ma.columns)>0 and pasada<2:
        a=ma.idxmax().values
        b=ma.columns
        rel=[]
        for j in range(len(a)):
            rel.append((a[j],b[j]))
        borrar,borrar_i=encontrar_relaciones(rel[:])
        ma=ma.drop(borrar,axis=1)
        #ma=ma.drop(borrar_i,axis=0)
        m_earth=m_earth.drop(borrar,axis=1)
        #m_earth=m_earth.drop(borrar_i,axis=0)
        m_mi=m_mi.drop(borrar,axis=1)
        #m_mi=m_mi.drop(borrar_i,axis=0)
        b_col.extend(borrar)
        b_index.extend(borrar_i)
        # relaciones contrarias
        #PARA REVISAR SI EXISTEN RELACIONES CONTRARIAS A TRAVÉS DEL USO DE CONCEPNET
        #if len(ma.index)>0 and len(ma.columns)>0:
        if len(ma.columns)>0:
            a=ma.idxmax().values
            b=ma.columns
            rel=[]
            for j in range(len(a)):
                rel.append((a[j],b[j]))
            if pasada==0:
                borrar,borrar_i,d=encontrar_relaciones_contrarias(rel[:])
            else:
                borrar,borrar_i,d1=encontrar_relaciones_contrarias(rel[:])
            ma=ma.drop(borrar,axis=1)
            ma=ma.drop(borrar_i,axis=0)
            #m_earth=m_earth.drop(borrar,axis=1)
            #m_earth=m_earth.drop(borrar_i,axis=0)
            m_mi=m_mi.drop(borrar,axis=1)
            #m_mi=m_mi.drop(borrar_i,axis=0)
            b_col.extend(borrar)
            b_index.extend(borrar_i)
        pasada+=1

    # ultima VUELTA PARA CHECAR RELACIONES CERCANAS --------------------------------------------------
    # relaciones cercanas quitar
    r_l2=[]
    if len(ma.index)>0 and len(ma.columns)>0:
        indexes=ma.index
        columnas=ma.columns
        borrar,borrar_i,r_l2=encontrar_relaciones_cercanas(indexes,columnas)
        ma=ma.drop(borrar,axis=1)
        #ma=ma.drop(borrar_i,axis=0)
        m_earth=m_earth.drop(borrar,axis=1)
        m_earth=m_earth.drop(borrar_i,axis=0)
        m_mi=m_mi.drop(borrar,axis=1)
        #m_mi=m_mi.drop(borrar_i,axis=0)
        b_col.extend(borrar)
        b_col.extend(borrar)
        b_index.extend(borrar_i)
    #print(ma.index,ma.columns)
        
    #   ALMACENAMIENTO DE TODA LA INFORMACIÓN PROCESADA
    #alamacenado de resultados
    sumas.append(ma.min().sum())# cambie a minimo antes era suma
    if len(ma.columns)==0:
        max_info.append(0)
    else:
        max_info.append(ma.max().sum())#/(ma.shape[1]))) modificación1
    entropias.append(entropia(ma.round(2).values.flatten())) # cambie a 2 decimales
    m_distancia = obtener_distancia(t_vectors,h_vectors,t_clean,h_clean,b_col,b_index)
    
    m_earth=m_earth*m_distancia
    mearts.append(m_earth.max().sum())#suma de sumas lo cambie a max
    mutinf.append(m_mi.max().sum())#cambie de suma a max
    distancias.append(m_distancia.sum().sum()) #cambie de maximas a sumas
    
    if len(ma.columns)==0:
        diferencias.append(1)
    elif len(ma.columns)>0 and len(ma.index)==0:
        diferencias.append(0.5)
    elif len(ma.columns)>0 and len(ma.index)>0:
        diferencias.append(len(ma.columns)/len(ma.index))
    
    d2=[]   
    if d==[] and d1==[]:
        d2=[]
    else:
        d2=list(set(d+d1))
    r2=[]   
    if r_l2==[]:
        r2=[]
    else:
        r2=list(set(r_l2))
    list_antonimos.append(len(d2))
    listas_malign.append(ma)
    lista_anto.append(d2)
    lista_related.append(len(r2))
    lista_relatedT.append(r2)
    #print(ma)
    st=""
    sh=""
    for t__1 in ma.index:
        st=st+" "+str(t__1).split("{")[0]
    for t__2 in ma.columns:
        sh=sh+" "+str(t__2).split("{")[0]
    doc1 = nlp(st)
    doc2 = nlp(sh)
    #print(doc1, "<->", doc2, doc1.similarity(doc2))
    #print(st,sh)
    if sh!="" and st!="":
        similitud_faltantes.append(doc1.similarity(doc2))
    elif st!="" and sh=="":
        similitud_faltantes.append(1)
    else:
        similitud_faltantes.append(0)
fin = time.time()
#clases=prueba["gold_label"].values
#temp1 =np.array([sumas,distancias,entropias,mutinf,mearts,max_info,list_antonimos,lista_related,similitud_faltantes])#,clases])
#df_resultados = pd.DataFrame(temp1.T,columns=["suma","distancias","entropias","mutual_info","m_earth","max_info_p","antonimos","relaciones","sim_faltantes"])#,"CLASS"])
#df_resultados.to_csv("salida/resultados/RTE3/"+archivo,index=False)
#df_resultados.to_csv("salida/resultados/"+sys.argv[1]+".csv",index=False)


print("Tiempo que se llevo:",round(fin-inicio,2)," segundos")

0
[[ 0.          0.29334688  0.04616581  0.36109045  0.        ]
 [ 0.         -0.02193861  0.542705    0.08267934  0.        ]
 [ 0.          1.          0.1593052   0.08208503  0.        ]
 [ 0.          0.22742113  0.3382341   0.1377191   0.        ]
 [ 0.          0.08208503 -0.12337436  1.          0.        ]
 [ 0.         -0.05488963  0.24554554  0.16763932  0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.03150407  0.11398686 -0.08131137  0.        ]
 [ 0.         -0.20003405  0.10279238 -0.13424754  0.        ]
 [ 0.          0.03639128  0.2253143   0.12864953  0.        ]
 [ 0.          0.01360783  0.09853348  0.20837757  0.        ]
 [ 0.          0.11394621  0.01404482  0.24989122  0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.1593052   0.08208503  0.        ]
 [ 0.          0.5007192   0.28838244  0.2702214   0.

In [37]:
ma

""
"1856{1856,NUM}"
"manuel{manuel,NOUN}"
"ruiz{ruiz,NOUN}"
"elected{elect,VERB}"
"deputy{deputy,NOUN}"
"soon{soon,ADV}"
"attracted{attract,VERB}"
"notice{notice,NOUN}"
"most{most,ADV}"
"advanced{advanced,ADJ}"


In [38]:
m_mi

""
"1856{1856,NUM}"
"manuel{manuel,NOUN}"
"ruiz{ruiz,NOUN}"
"elected{elect,VERB}"
"deputy{deputy,NOUN}"
"soon{soon,ADV}"
"attracted{attract,VERB}"
"notice{notice,NOUN}"
"most{most,ADV}"
"advanced{advanced,ADJ}"


In [39]:
m_earth

""
"1856{1856,NUM}"
"manuel{manuel,NOUN}"
"ruiz{ruiz,NOUN}"
"elected{elect,VERB}"
"deputy{deputy,NOUN}"
"soon{soon,ADV}"
"attracted{attract,VERB}"
"notice{notice,NOUN}"
"most{most,ADV}"
"advanced{advanced,ADJ}"


In [40]:
m_earth.idxmin()

Series([], dtype: object)